In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [9]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'Regular_Array'

# MakeModule Parameters
moduletype=simulationname
numpanels = 2
x = 2  # 2-up landscape
y = 1
xgap = 0.5
ygap = 0.5
zgap = 0.1
sensorsy = 9 


# TorqueTube Parameters
axisofrotationTorqueTube=True
torqueTube = True
tubetype = 'square'
diameter = 0.1
torqueTubeMaterial = 'Metal_Grey'       # IT's NOT GRAY, IT's GREY
cellLevelModule = False
numcellsx=10
numcellsy=6

# SceneDict Parameters
gcr = 0.33   # ground cover ratio,  = module_height / pitch
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 0
nMods = 4
nRows =3

azimuth_ang=90
tilt = 30
axis_tilt = 30

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Noon, June 17th

# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                           material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                           rewriteModulefile = True, xgap=xgap, 
                           axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                           numcellsx=numcellsx, numcellsy = numcellsy)
# create a scene with all the variables


path = C:\Users\sayala\Documents\RadianceScenes\Demo3
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'Regular_Array')
REWRITING pre-existing module file. 
Module Regular_Array successfully created


In [10]:
sceneDict = {'tilt':tilt,'pitch': 3,'height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname, 'nMods': nMods, 'nRows': nRows, 'axis_tilt': axis_tilt}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


Created Regular_Array.oct


In [4]:
name='Mycube'
text='! genbox black cuteBox 0.05 0.05 7 | xform -t -0.025 -0.025 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')
'''
name='Mycube2'
text='! genbox black cuteBox 0.1 0.1 2.5 | xform -t 3 -1.491 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')
'''
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.


('\nCustom Object Name', 'objects\\Mycube.rad')
Created Regular_Array.oct


At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:

# rvu -vf views\front.vp -e .01 Regular_Array.oct


In [11]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
modWanted = 2
rowWanted = 2
frontscan, backscan = analysis.moduleAnalysis(scene, modWanted=modWanted, rowWanted=rowWanted)
analysis.analysis(octfile, "test3", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )

(0, 90, 0, 4, 3, 3, 0.15000000000000002, 2.5, 2.5)
fixing height for axis_tilt
Linescan in process: test3_Front
Linescan in process: test3_Back
Saved: results\irr_test3.csv
Annual bifacial ratio average:  0.361


In [ ]:
scene.sceneDict

In [8]:
frontscan

{'Nx': 1,
 'Ny': 9.0,
 'Nz': 1,
 'orient': '0 0 -1',
 'xinc': -0.21650635094610968,
 'xstart': 0.9410254037844387,
 'yinc': -1.3257190484061336e-17,
 'ystart': 5.762118743304792e-17,
 'zinc': 0.12499999999999999,
 'zstart': 1.4899038105676656}